<a href="https://colab.research.google.com/github/rubyvanrooyen/NIFTyworkshop/blob/master/Example_RMdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from matplotlib import pylab
import io
import numpy
import numpy as np
import requests

In [ ]:
# example input signal (1D with power spectrum)
url='https://raw.githubusercontent.com/rubyvanrooyen/NIFTyworkshop/master/PDATA-RMSYN-1.TXT'
res = requests.get(url)
f = io.StringIO(res.text)
data = f.readlines()

In [ ]:
# four columns in each file: frequencies in  Hz, Stokes Q , Stokes U and Stokes I in Jansky per beam
freq_hz = []
stokes_q = []
stokes_u = []
stokes_i = []
for line in data:
    [freq, q, u, i] = line.strip().split()
    freq_hz.append(freq)
    stokes_q.append(q)
    stokes_u.append(u)
    stokes_i.append(i)
freq_hz = np.array(freq_hz, dtype=float)
stokes_q = np.array(stokes_q, dtype=float)
stokes_u = np.array(stokes_u, dtype=float)
stokes_i = np.array(stokes_i, dtype=float)

In [ ]:
print(freq_hz.shape)
print(stokes_q.shape)
print(stokes_u.shape)
print(stokes_i.shape)


In [ ]:
# fractional polarisation
# P = pI = Q + iU
Pcorr = stokes_q + 1j*stokes_u
p = Pcorr/stokes_i

In [ ]:
def faraday_synthesis(x, pol, phi_range, lambda0=False):
    f = numpy.zeros(len(phi_range), dtype=complex)

    N = len(x)
    xmean = x.mean()

    for k, phi in enumerate(phi_range):
        if lambda0:
            f[k] = numpy.sum(pol * numpy.exp(-2j * phi * (x-xmean) ))/N
        else:
            f[k] = numpy.sum(pol * numpy.exp(-2j * phi * x ))/N
                    
    return f

In [ ]:
# c = Speed of Light (299,792,458 m/s)
# define lambda^2
dx = 3.835e-5  # m^2 
# x = numpy.arange((3.0e8/freq_hz[-1])**2, (3.0e8/freq_hz[0])**2, dx)  # m^2
x = (3.0e8/freq_hz)**2  # m^2

# Faraday depth
# define Faraday depth range
phi_min = -10000
phi_max = 10000
d_phi = (phi_max - phi_min)/float(len(P))
phi_range = numpy.arange(phi_min, phi_max, d_phi)

In [ ]:
# Faraday spectrum
rmtf = faraday_synthesis(x,  # wavelength: wavelength squared.
                         p,
                         phi_range,  # Faraday depth range
                         lambda0=False)

In [ ]:

#plot fractional polarisation vs wavelength
# fractional polarisation (P/I) amplitude
fig, axs = pylab.subplots(1, 3, figsize=(15, 3), facecolor='white')
axs[0].plot(x, np.abs(p))
axs[0].set_xlabel(r'Wavelength [m$^2$]')
axs[0].set_ylabel(r'Fractional polarisation')

# plot polarisation angle (lambda**2) [rad] vs wavelength [m**2]
#observed position angle (black) and the residual position angle
#(blue) after the dominant peak in the Faraday spectrum was removed.
pol_angle = 0.5*np.unwrap(np.angle(P))
#pol_angle = 0.5*np.unwrap(np.angle(p))
axs[1].plot(x, pol_angle - pol_angle.max())
axs[1].set_xlabel(r'Wavelength [m$^2$]')
axs[1].set_ylabel(r'Pol angle ($\lambda^2$) [rad]')

# Faraday spectrum vs faraday depth [rad/m**2]
axs[2].plot(phi_range, np.abs(rmtf)) 
axs[2].set_xlabel(r'Faraday depth [rad/m$^2$]')
axs[2].set_ylabel(r'Faraday spectrum')

pylab.show()

In [ ]:
#plot fractional polarisation vs wavelength
# fractional polarisation (P/I) amplitude
fig, axs = pylab.subplots(1, 2, figsize=(7, 3), facecolor='white')
# Faraday spectrum vs faraday depth [rad/m**2]
axs[0].plot(phi_range, np.abs(rmtf)) 
axs[0].set_xlabel(r'Faraday depth [rad/m$^2$]')
axs[0].set_ylabel(r'Faraday spectrum')

axs[1].plot(x, np.abs(numpy.fft.fftshift(numpy.fft.fft(rmtf)))) 

pylab.show()